In [1]:
import torch
from sklearn.metrics import accuracy_score, f1_score
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import gensim.downloader as api
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [2]:
# Load pre-trained word embeddings
word_vectors = api.load("word2vec-google-news-300")
glove_vectors = api.load("glove-wiki-gigaword-300")
fasttext_vectors = api.load("fasttext-wiki-news-subwords-300")

In [3]:
class RNNModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, word_embeddings):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(word_embeddings, freeze=True)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
        rnn_out, _ = self.rnn(embeds)
        tag_space = self.linear(rnn_out)
        return tag_space
    
class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, word_embeddings):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(word_embeddings, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.linear(lstm_out)
        return tag_space
    
class GRUModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, word_embeddings):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(word_embeddings, freeze=True)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
        gru_out, _ = self.gru(embeds)
        tag_space = self.linear(gru_out)
        return tag_space

In [4]:
word2vec_state_dict = torch.load('t2_model1_word2vec.pt')

In [5]:
glove_state_dict = torch.load('t2_model1_GloVe.pt')

In [6]:
fasttext_state_dict = torch.load('t2_model1_fasttext.pt')

In [7]:
word2vec_state_dict2 = torch.load('t2_model2_word2vec.pt')

In [8]:
glove_state_dict2 = torch.load('t2_model2_GloVe.pt')

In [9]:
fasttext_state_dict2 = torch.load('t2_model2_fasttext.pt')

In [10]:
word2vec_state_dict3 = torch.load('t2_model3_word2vec.pt')

In [11]:
glove_state_dict3 = torch.load('t2_model3_GloVe.pt')

In [12]:
fasttext_state_dict3 = torch.load('t2_model3_fasttext.pt')

In [13]:
word2vec_model = RNNModel(word_vectors.vector_size, 128, 3, torch.FloatTensor(word_vectors.vectors))
glove_model = RNNModel(glove_vectors.vector_size, 128, 3, torch.FloatTensor(glove_vectors.vectors))
fasttext_model = RNNModel(fasttext_vectors.vector_size, 128, 3, torch.FloatTensor(fasttext_vectors.vectors))

word2vec_model2 = LSTMModel(word_vectors.vector_size, 128, 3, torch.FloatTensor(word_vectors.vectors))
glove_model2 = LSTMModel(glove_vectors.vector_size, 128, 3, torch.FloatTensor(glove_vectors.vectors))
fasttext_model2 = LSTMModel(fasttext_vectors.vector_size, 128, 3, torch.FloatTensor(fasttext_vectors.vectors))

word2vec_model3 = GRUModel(word_vectors.vector_size, 128, 3, torch.FloatTensor(word_vectors.vectors))
glove_model3 = GRUModel(glove_vectors.vector_size, 128, 3, torch.FloatTensor(glove_vectors.vectors))
fasttext_model3 = GRUModel(fasttext_vectors.vector_size, 128, 3, torch.FloatTensor(fasttext_vectors.vectors))

In [14]:
word2vec_model.load_state_dict(word2vec_state_dict)

<All keys matched successfully>

In [15]:
glove_model.load_state_dict(glove_state_dict)

<All keys matched successfully>

In [16]:
fasttext_model.load_state_dict(fasttext_state_dict)

word2vec_model2.load_state_dict(word2vec_state_dict2)
glove_model2.load_state_dict(glove_state_dict2)
fasttext_model2.load_state_dict(fasttext_state_dict2)

word2vec_model3.load_state_dict(word2vec_state_dict3)
glove_model3.load_state_dict(glove_state_dict3)
fasttext_model3.load_state_dict(fasttext_state_dict3)

<All keys matched successfully>

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score
import json

def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def prepare_data(data, word_to_idx, label_to_idx):
    X, y = [], []
    for sample in data.values():
        text = sample['text'].split()
        labels = sample['labels']
        text_indices = [word_to_idx.get(word, 0) for word in text]
        X.append(torch.tensor(text_indices))
        y.append(torch.tensor([label_to_idx.get(label, -1) for label in labels if label in label_to_idx]))
    X_padded = pad_sequence(X, batch_first=True, padding_value=0)
    y_padded = pad_sequence(y, batch_first=True, padding_value=0)
    return X_padded, y_padded

test_data = load_dataset('ATE_test.json')

word_to_idx_word2vec = {word: idx + 1 for idx, word in enumerate(word_vectors.index_to_key)}
word_to_idx_glove = {word: idx + 1 for idx, word in enumerate(glove_vectors.index_to_key)}
word_to_idx_fasttext = {word: idx + 1 for idx, word in enumerate(fasttext_vectors.index_to_key)}
label_to_idx = label_to_ix = {'B':0, 'I':1, 'O':2}

In [19]:
def evaluate_model2(model, dataloader, tagset_size):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for text, labels in dataloader:
            outputs = model(text)
            loss = criterion(outputs.view(-1, tagset_size), labels.view(-1))
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 2)
            all_preds.extend(predicted.view(-1).cpu().numpy().tolist())
            all_labels.extend(labels.view(-1).cpu().numpy().tolist())
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    accuracy = accuracy_score(all_labels, all_preds)

    return accuracy, macro_f1

X_test_word2vec, y_test_word2vec = prepare_data(test_data, word_to_idx_word2vec, label_to_idx)
test_dataset_word2vec = TensorDataset(X_test_word2vec, y_test_word2vec)
test_loader_word2vec = DataLoader(test_dataset_word2vec, batch_size=32, shuffle=True)

word2vec_accuracy, word2vec_f1 = evaluate_model2(word2vec_model, test_loader_word2vec, 3)
word2vec_accuracy2, word2vec_f12 = evaluate_model2(word2vec_model2, test_loader_word2vec, 3)
word2vec_accuracy3, word2vec_f13 = evaluate_model2(word2vec_model3, test_loader_word2vec, 3)


X_test_glove, y_test_glove = prepare_data(test_data, word_to_idx_glove, label_to_idx)
test_dataset_glove = TensorDataset(X_test_glove, y_test_glove)
test_loader_glove = DataLoader(test_dataset_glove, batch_size=32, shuffle=True)

glove_accuracy, glove_f1 = evaluate_model2(glove_model, test_loader_glove, 3)
glove_accuracy2, glove_f12 = evaluate_model2(glove_model2, test_loader_glove, 3)
glove_accuracy3, glove_f13 = evaluate_model2(glove_model3, test_loader_glove, 3)


X_test_fasttext, y_test_fasttext = prepare_data(test_data, word_to_idx_fasttext, label_to_idx)
test_dataset_fasttext = TensorDataset(X_test_fasttext, y_test_fasttext)
test_loader_fasttext = DataLoader(test_dataset_fasttext, batch_size=32, shuffle=True)

fasttext_accuracy, fasttext_f1 = evaluate_model2(fasttext_model, test_loader_fasttext, 3)
fasttext_accuracy2, fasttext_f12 = evaluate_model2(fasttext_model2, test_loader_fasttext, 3)
fasttext_accuracy3, fasttext_f13 = evaluate_model2(fasttext_model3, test_loader_fasttext, 3)

# Print the results
print("T2 Model 1 Word2Vec Test Accuracy:", word2vec_accuracy, " Test F1 Score:", word2vec_f1)
print("T2 Model 1 GloVe Test Accuracy:", glove_accuracy, " Test F1 Score:", glove_f1)
print("T2 Model 1 Fasttext Test Accuracy:", fasttext_accuracy, " Test F1 Score:", fasttext_f1)
print("T2 Model 2 Word2Vec Test Accuracy:", word2vec_accuracy2, " Test F1 Score:", word2vec_f12)
print("T2 Model 2 GloVe Test Accuracy:", glove_accuracy2, " Test F1 Score:", glove_f12)
print("T2 Model 2 Fasttext Test Accuracy:", fasttext_accuracy2, " Test F1 Score:", fasttext_f12)
print("T2 Model 3 Word2Vec Test Accuracy:", word2vec_accuracy3, " Test F1 Score:", word2vec_f13)
print("T2 Model 3 GloVe Test Accuracy:", glove_accuracy3, " Test F1 Score:", glove_f13)
print("T2 Model 3 Fasttext Test Accuracy:", fasttext_accuracy3, " Test F1 Score:", fasttext_f13)

T2 Model 1 Word2Vec Test Accuracy: 0.9636293369975953  Test F1 Score: 0.6418273258905921
T2 Model 1 GloVe Test Accuracy: 0.9705427688079697  Test F1 Score: 0.7345606849465055
T2 Model 1 Fasttext Test Accuracy: 0.9688680865681897  Test F1 Score: 0.642376971497277
T2 Model 2 Word2Vec Test Accuracy: 0.9644881484026108  Test F1 Score: 0.6326217148585592
T2 Model 2 GloVe Test Accuracy: 0.9692545517004466  Test F1 Score: 0.643781609162235
T2 Model 2 Fasttext Test Accuracy: 0.968610443146685  Test F1 Score: 0.6368809473533991
T2 Model 3 Word2Vec Test Accuracy: 0.965218138096874  Test F1 Score: 0.6331449380684212
T2 Model 3 GloVe Test Accuracy: 0.9709721745104775  Test F1 Score: 0.7176116417133045
T2 Model 3 Fasttext Test Accuracy: 0.9688680865681897  Test F1 Score: 0.6371674868854306
